In [42]:
import numpy as np
import matplotlib.pyplot as plt 

import pandas as pd  
import seaborn as sns 

%matplotlib inline

In [48]:
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

In [75]:
data = pd.read_csv("housesalesprediction/kc_house_data.csv")
data.shape

(21613, 21)

In [51]:
feature_cols = ['bedrooms', 'bathrooms', 'sqft_living','sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
             'sqft_above', 'sqft_basement','sqft_living15', 'sqft_lot15']
target_col = 'price'

In [53]:
X = data[feature_cols]
Y = data[target_col]

In [54]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=5)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(17290, 13)
(4323, 13)
(17290,)
(4323,)


In [55]:
# import the regressor 
from sklearn.tree import DecisionTreeRegressor  
  
# create a regressor object 
regressor = DecisionTreeRegressor(random_state = 0, min_samples_leaf = 20, max_depth = 4)
  
# fit the regressor with X and Y data
regressor.fit(X_train, Y_train) 

DecisionTreeRegressor(criterion='mse', max_depth=4, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=20,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=0, splitter='best')

In [56]:
from sklearn.metrics import mean_squared_error

# model evaluation for training set
y_train_predict = regressor.predict(X_train)
rmse = (np.sqrt(mean_squared_error(Y_train, y_train_predict)))

print("The model performance for training set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))

# model evaluation for testing set
y_test_predict = regressor.predict(X_test)
rmse = (np.sqrt(mean_squared_error(Y_test, y_test_predict)))

print("The model performance for testing set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))

The model performance for training set
--------------------------------------
RMSE is 235274.95890555577
The model performance for testing set
--------------------------------------
RMSE is 235768.30892321272


In [57]:
df_train = pd.concat([X_train,Y_train], axis = 1)
df_train['Tree_pred'] = y_train_predict
df_train.loc[:,'Leaf_node'] = regressor.apply(X_train)

In [58]:
df_test = pd.concat([X_test,Y_test], axis = 1)
df_test['Tree_pred'] = y_test_predict
df_test.loc[:,'Leaf_node'] = regressor.apply(X_test)

In [60]:
feature_cols = ['bedrooms', 'bathrooms', 'sqft_living','sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
             'sqft_above', 'sqft_basement','sqft_living15', 'sqft_lot15']
target_col = 'price'

In [61]:
from sklearn.neighbors import KNeighborsRegressor

df_final = pd.DataFrame()
for i in df_test['Leaf_node'].unique():
    X_train = df_train[df_train['Leaf_node']==i]
    X_test = df_test[df_test['Leaf_node']==i]
    
    if(X_train.shape[0]<5):
        X_test.loc[:,'pred'] = regressor.predict(X_test[feature_cols])
        
    else:
        neigh = KNeighborsRegressor(n_neighbors=3)
        neigh.fit(X_train[feature_cols],X_train[target_col])
        X_test.loc[:,'pred'] = neigh.predict(X_test[feature_cols])
    df_final = pd.concat([df_final,X_test], axis = 0)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [62]:
df_final.head(3)

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,sqft_living15,sqft_lot15,price,Tree_pred,Leaf_node,pred
17485,4,2.5,2800,6820,2.0,0,0,4,7,2800,0,2060,6820,365000.0,590540.557647,12,594333.333333
15262,4,2.0,2680,18768,1.0,0,0,5,8,2680,0,1230,15750,360000.0,590540.557647,12,601500.000000
21496,4,2.5,3040,6425,2.0,0,0,3,8,3040,0,3040,7800,776000.0,590540.557647,12,654000.000000


In [63]:
(np.sqrt(mean_squared_error(df_final[target_col], df_final['Tree_pred'])))

235768.30892321272

In [64]:
(np.sqrt(mean_squared_error(df_final[target_col], df_final['pred'])))

242869.2138218758

In [65]:
# MAPE
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [66]:
mean_absolute_percentage_error(df_final[target_col], df_final['Tree_pred'])

32.809377402733816

In [67]:
mean_absolute_percentage_error(df_final[target_col], df_final['pred'])

31.852058711376234

In [74]:
df_final['Tree_pred'].std(), df_final['pred'].std()

(287864.8475659553, 320870.29090785043)

### Random Forest

In [68]:
data.head(3)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062


In [69]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=5)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(17290, 13)
(4323, 13)
(17290,)
(4323,)


In [70]:
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor(max_depth=3, random_state=0,
                             n_estimators=100)
regr.fit(X_train, Y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=3,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [71]:
from sklearn.metrics import mean_squared_error

# model evaluation for training set
y_train_predict = regr.predict(X_train)
rmse = (np.sqrt(mean_squared_error(Y_train, y_train_predict)))

print("The model performance for training set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))

# model evaluation for testing set
y_test_predict = regr.predict(X_test)
rmse = (np.sqrt(mean_squared_error(Y_test, y_test_predict)))

print("The model performance for testing set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))

print('MAPE for testing set is')
mean_absolute_percentage_error(Y_test, y_test_predict)

The model performance for training set
--------------------------------------
RMSE is 234874.94184728496
The model performance for testing set
--------------------------------------
RMSE is 236554.72318932947
MAPE for testing set is


33.573719423596536